# Progressive read size evaluation

### Using the 2.2GB sra_data.fastq data file.
We know we can do 10k reads so let's take progressively larger number of reads to show a progression of processing time.

In [1]:
from pyspark import SparkContext

In [2]:
import subprocess
import os

In [3]:
import eulercuda as ec

In [4]:
import pycuda.driver
import pycuda.autoinit

In [5]:
from collections import OrderedDict

In [ ]:
from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure


In [7]:
datafile = '/home/ubuntu/genome/sra_clean'

In [8]:
output_notebook()
# output_file("results.html")

Loading BokehJS ...

In [9]:
def make_dataset(read_size):
    dataset = []
    with open (datafile, 'r') as infile:
        for i in range(read_size):
            line = infile.readline().rstrip()
            if 'N' not in line:
                dataset.append(line)
    return dataset

In [10]:
def hail_mary(path, data):
    import eulercuda.eulercuda as ec
    subprocess.call(["hdfs", "dfs", "-rm", "-r", "-f", '/genome/' + path])
    hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, buffer=x, readLength = dataLength,readCount=dataCount)) \
        .saveAsTextFile('hdfs://172.31.26.32/genome/' + path)

In [11]:
timing_data = OrderedDict()
increment = 5000
read_size = 5000
dataset = []

In [15]:
k = 17
lmerLength = 18
run_ok = True
while run_ok:
    dataset = make_dataset(read_size)
    dataLength = len(dataset[0])
    rdd_data = sc.parallelize(dataset, (read_size // 2000))
    dataCount = rdd_data.count() // rdd_data.getNumPartitions()
    try:
        time = %timeit -o -r 5 hail_mary('ec'+str(read_size)[:2]+'k_output',rdd_data)
    except:
        print('Failure at '+ str(read_size), flush=True)
        run_ok = False
    if run_ok:
        timing_data[read_size] = time
        print(read_size,time.best, flush=True)
        print()
        read_size += increment

1 loop, best of 5: 23.6 s per loop
40000 23.575782978999996

1 loop, best of 5: 29.6 s per loop
45000 29.55098228299994

1 loop, best of 5: 30 s per loop
50000 29.988303688000087

1 loop, best of 5: 30.6 s per loop
55000 30.58005624999987

1 loop, best of 5: 35.3 s per loop
60000 35.32120414399924

1 loop, best of 5: 36.9 s per loop
65000 36.94936921599947

1 loop, best of 5: 37.5 s per loop
70000 37.49704381799984

1 loop, best of 5: 43.5 s per loop
75000 43.49610917800055

1 loop, best of 5: 44.1 s per loop
80000 44.06622747599977

1 loop, best of 5: 45.2 s per loop
85000 45.24880558199948

Failure at 90000


In [16]:
timing_data.keys()

odict_keys([5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000])

In [29]:
upper_limit = next(reversed(timing_data))

In [18]:
run_times = [v.best for v in timing_data.values()]
read_sizes = [k//1000 for k in timing_data.keys()]

In [19]:
read_sizes

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]

In [20]:
run_times

[11.267709831000047,
 9.818606054000156,
 10.435634698000285,
 15.91144412299991,
 17.07000417299969,
 21.101019450999956,
 23.40265847699993,
 23.575782978999996,
 29.55098228299994,
 29.988303688000087,
 30.58005624999987,
 35.32120414399924,
 36.94936921599947,
 37.49704381799984,
 43.49610917800055,
 44.06622747599977,
 45.24880558199948]

In [22]:
# p = None
p = figure(plot_width=800, plot_height=400, title="GPU Assisted Assembly")
p.xaxis.axis_label='Read Size'
p.yaxis.axis_label='Time'
p.line(read_sizes,run_times, line_width=1)
show(p)

## Now let's do the same thing with the pure python assembler

In [23]:
# datafile = '/home/ubuntu/genome/ba100k_clean.txt'
datafile = '/home/ubuntu/genome/sra_clean'

In [24]:
compliment = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

In [25]:
def twin(km):
    compliment = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    # return Seq.reverse_complement(km)
    return "".join(compliment.get(base, base) for base in reversed(km))

def fw(km):
    for x in 'ACGT':
        yield km[1:]+x

def bw(km):
    for x in 'ACGT':
        yield x + km[:-1]

In [26]:
def contig_to_string(c):
    return c[0] + ''.join(x[-1] for x in c[1:])

def get_contig(d,km):
    '''
    Find kmer's contig.
    Return: the string, list of kmers in contig
    '''
    c_fw = get_contig_forward(d,km)

    c_bw = get_contig_forward(d,twin(km))

    if km in fw(c_fw[-1]):
        c = c_fw
    else:
        c = [twin(x) for x in c_bw[-1:0:-1]] + c_fw
    return contig_to_string(c),c


def get_contig_forward(d,km):
    c_fw = [km]

    while True:
        if sum(x in d for x in fw(c_fw[-1])) != 1:
            break

        cand = [x for x in fw(c_fw[-1]) if x in d][0]
        if cand == km or cand == twin(km):
            break # break out of cycles or mobius contigs
        if cand == twin(c_fw[-1]):
            break # break out of hairpins

        if sum(x in d for x in bw(cand)) != 1:
            break

        c_fw.append(cand)

    return c_fw

def all_contigs(k_tuples):
    d = dict(k_tuples)
    done = set()
    r = []
    for x in d:
        if x not in done:
            s,c = get_contig(d,x)
            for y in c:
                done.add(y)
                done.add(twin(y))
            r.append(s)
    return r

In [27]:
def do_baseline(run_size, path):
    k = 17
    lmerLength = 18
    subprocess.call(["hdfs", "dfs", "-rm", "-r", "-f", '/genome/' + path])
    dataset = make_dataset(run_size)
    data = sc.parallelize(dataset, (run_size // 2000))
    fwd_list = data.flatMap(lambda x: [x[i:i+k] for i in range(len(x.rstrip())-k+1)])
    rev_comp = data.map(lambda x:''.join(reversed([compliment.get(base, base) for base in x])))
    rev_list = rev_comp.flatMap(lambda x: [x[i:i+k] for i in range(len(x.rstrip())-k+1)])
    kmer_list = fwd_list + rev_list
    emitter = kmer_list.map(lambda x: (x, 1))
    kmer_counts = emitter.reduceByKey(lambda x, y: x+y)
    asm_time = kmer_counts.mapPartitions(all_contigs).saveAsTextFile('hdfs://172.31.26.32/genome/' + path)
    

In [30]:
#upper_limit = 60000
upper_limit += 5000

In [31]:
base_timing = OrderedDict()

In [ ]:
#do_baseline(10000,'base_output')

In [32]:
for i in range(5000, upper_limit, 5000):
#     print(str(i), ' reads: ', flush=True)
    timing = %timeit -o -r 5 do_baseline(i, "base_output")
    print(i, timing.best, flush=True)
    print(flush=True)
    base_timing[i] = timing

1 loop, best of 5: 5.85 s per loop
5000 5.848415021999244

1 loop, best of 5: 13.4 s per loop
10000 13.375951134000388

The slowest run took 4.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 8.38 s per loop
15000 8.383503173000463

1 loop, best of 5: 10.2 s per loop
20000 10.200104548999661

1 loop, best of 5: 12.8 s per loop
25000 12.840071923999858

1 loop, best of 5: 14.6 s per loop
30000 14.59803893499975

1 loop, best of 5: 15.4 s per loop
35000 15.409415425000589

1 loop, best of 5: 17.2 s per loop
40000 17.226859213000353

1 loop, best of 5: 19.3 s per loop
45000 19.325862753000365

1 loop, best of 5: 20.9 s per loop
50000 20.909636643999875

1 loop, best of 5: 21.9 s per loop
55000 21.92055673699997

1 loop, best of 5: 24 s per loop
60000 23.98209688099996

1 loop, best of 5: 25.7 s per loop
65000 25.746402654999656

1 loop, best of 5: 26.4 s per loop
70000 26.41261695999947

1 loop, best of 5: 28.2 s per loop
75

In [33]:
base_run_times = [v.best for v in base_timing.values()]
base_read_sizes = [k//1000 for k in base_timing.keys()]

In [34]:
base_read_sizes

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]

In [35]:
base_run_times

[5.848415021999244,
 13.375951134000388,
 8.383503173000463,
 10.200104548999661,
 12.840071923999858,
 14.59803893499975,
 15.409415425000589,
 17.226859213000353,
 19.325862753000365,
 20.909636643999875,
 21.92055673699997,
 23.98209688099996,
 25.746402654999656,
 26.41261695999947,
 28.197112466000362,
 29.624492800000553,
 30.450817831000677]

In [36]:
p = figure(plot_width=800, plot_height=400, title="Pure Python Assembly")
p.xaxis.axis_label='Read Size'
p.yaxis.axis_label='Time'
p.line(base_read_sizes,base_run_times,color="navy",line_width=1)
show(p)

In [37]:
myPlot = figure(plot_width=800, plot_height=400, title="GPU Assisted Assembly vs Pure Python Assembly")
myPlot.xaxis.axis_label='Read Size'
myPlot.yaxis.axis_label='Time'
myPlot.multi_line([read_sizes,run_times],[base_run_times,base_read_sizes],color=["firebrick", "navy"],line_width=2)
show(myPlot)

In [39]:
p = figure(plot_width=800, plot_height=400, title="GPU Assisted Assembly vs Pure Python Assembly")
p.xaxis.axis_label='Read Size'
p.yaxis.axis_label='Time'
p.circle(read_sizes,run_times, legend='GPU Assisted', color='navy')
p.line(read_sizes,run_times, legend='GPU Assisted',line_width=2)
p.square(base_read_sizes,base_run_times,legend='Pure Python',color="firebrick")
p.line(base_read_sizes,base_run_times,legend='Pure Python',color="firebrick",line_width=2)
p.legend.location='bottom_right'
show(p)